In [ ]:
import pandas as pd
import numpy as np
import os
import re
import datetime
from skimpy import clean_columns
import plotly.express as px
import plotly.graph_objects as go

# Navigate to data folder
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, 'data')
files = os.listdir(data_directory)
if not files:
    raise FileNotFoundError("No files found in the data directory.")
file = files[0]
file_path = os.path.join(data_directory, file)

# Get list of sheets in the Excel file
list_sheets = pd.ExcelFile(file_path).sheet_names

# Read Excel file
df_data = pd.read_excel(file_path, sheet_name=None)

# Initialize empty dataframe
df_holdings = pd.DataFrame()

# Loop through the first two sheets and concatenate them
for i in range(len(list_sheets) - 1):
    df = df_data[list_sheets[i]]
    match = re.search(r'\((\d{1,2} \w+ \d{4})\)', list_sheets[i])
    if match:
        date_str = match.group(1)
        df = df.assign(datestamp=pd.to_datetime(date_str, format='%d %B %Y'))
    df_holdings = pd.concat([df_holdings, df], ignore_index=True)

# Assign the last sheet to df_risk_metrics
df_risk_metrics = df_data[list_sheets[-1]]

# clean column names
df_holdings = clean_columns(df_holdings)
df_risk_metrics = clean_columns(df_risk_metrics)

# Remove rows where portfolio_code/asset_id is null
df_holdings.dropna(subset=['asset_id'], inplace=True)
df_risk_metrics.dropna(subset=['portfolio_code'], inplace=True)

# Extract March and June data
df_june = df_holdings[df_holdings['datestamp'] == pd.Timestamp('2024-06-30')]
df_march = df_holdings[df_holdings['datestamp'] == pd.Timestamp('2024-03-31')]

# KDE Plot with Plotly
fig_kde = go.Figure()
fig_kde.add_trace(go.Histogram(x=df_june["weight_%"], name="June 2024", opacity=0.5, histnorm='probability density'))
fig_kde.add_trace(go.Histogram(x=df_march["weight_%"], name="March 2024", opacity=0.5, histnorm='probability density'))
fig_kde.update_layout(title="Portfolio Weight Distribution: March vs. June 2024", xaxis_title="Portfolio weight_%", yaxis_title="Density", barmode='overlay')
fig_kde.show()

# Compute weight change - TODO: Fix this
# weight_change = df_june.set_index("asset_id")["weight_%"] - df_march.set_index("asset_id")["weight_%"]
# top_weight_changes = weight_change.abs().nlargest(10)

# # Bar Plot for Weight Change
# fig_weight = px.bar(x=top_weight_changes.index, y=top_weight_changes.values, labels={'x': 'Asset ID', 'y': 'Weight Change (%)'}, title="Top 10 Largest Weight Changes (March vs. June)", color=top_weight_changes.values, color_continuous_scale='coolwarm')
# fig_weight.show()

# Compute active risk change
risk_change = df_june.set_index("asset_id")["active_total_risk"] - df_march.set_index("asset_id")["active_total_risk"]
top_risk_changes = risk_change.abs().nlargest(10)

# Bar Plot for Risk Change
fig_risk = px.bar(x=top_risk_changes.index, y=top_risk_changes.values, labels={'x': 'Asset ID', 'y': 'Active Risk Change'}, title="Top 10 Largest Active Risk Changes (March vs. June)", color=top_risk_changes.values, color_continuous_scale='magma')
fig_risk.show()

# Compute duration change
duration_change = df_june.set_index("asset_id")["active_effective_duration_mac"] - df_march.set_index("asset_id")["active_effective_duration_mac"]
top_duration_changes = duration_change.abs().nlargest(10)

# Bar Plot for Duration Change
fig_duration = px.bar(x=top_duration_changes.index, y=top_duration_changes.values, labels={'x': 'Asset ID', 'y': 'Duration Change'}, title="Top 10 Largest Duration Changes (March vs. June)", color=top_duration_changes.values, color_continuous_scale='viridis')
fig_duration.show()

# Line Plot for Tracking Error
fig_tracking = px.line(df_risk_metrics, x="reference_date", y="tracking_error_ex_ante", title="Tracking Error Over Time", markers=True)
fig_tracking.show()

# Line Plot for Credit Spread Duration
fig_credit_spread = px.line(df_risk_metrics, x="reference_date", y="credit_spread_dur_active", title="Credit Spread Duration Over Time", markers=True, color_discrete_sequence=['red'])
fig_credit_spread.show()
